In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import nltk
import regex
import urllib
import gensim
from gensim.utils import lemmatize
from gensim.utils import simple_preprocess
import spacy 
from spacy.lang.it.stop_words import STOP_WORDS as ita
from wordcloud import WordCloud
from matplotlib import pyplot as plt


In [ ]:
!python3 -m spacy download it_core_news_sm

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/data_processed_ITA_MAY_26.csv')
type(df.Date[0])
for c in range(len(df)):
    li = list(str(df.Date[c]).split(" "))
    for n, i in enumerate(li):
        if i == 'gennaio':
            li[n] = 'January'
        if i == 'febbraio':
            li[n] = 'February'
        if i == 'marzo':
            li[n] = 'March'
        if i == 'aprile':
            li[n] = 'April'
        if i == 'maggio':
            li[n] = 'May'
    df.Date[c] = ", ".join(li)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/ita2020.csv')
df['Date'] =pd.to_datetime(df.Date)
df =df.dropna().set_index('Date').sort_index()

texts = df.Article.values.tolist()
# df['weekoftheyear'] = [week.weekofyear for week in df.index]
print(len(df))
df.tail()

8175


,Title,Article
Date,,
2020-07-14,L’età dell’Odio in rete e la ricerca tribale d...,Perché ogni giorno internet sembra darci in pa...
2020-07-14,Il lavoro (quando c’è) fa scattare il richiamo...,I lavoratori fanno la fila per i posti di lavo...
2020-07-14,"Bmw iX3, ecco il primo suv elettrico bavarese",La iX3 apre la strada alla nuova generazione d...
2020-07-14,"Banche Usa, cadono gli utili per le maxi-riser...",Profitti in caduta da coronavirus per le grand...
2020-07-14,"Rivoli, dopo il lockdown le misure per tornare...","Ingressi contingentati, norme comportamentali ..."


In [ ]:
df = df.assign(weekoftheyear=pd.PeriodIndex(df.index, freq='M'))
df

,Title,Article,weekoftheyear
Date,,,
2020-01-01,Europa e la cultura: il bilancio 2019,Il Parlamento Europeo ha recentemente sospeso ...,2020-01
2020-01-01,L’anno dei “lateral hire”: quando il socio del...,Il 2019 è stato un anno record per quanto rigu...,2020-01
2020-01-01,"Bollette, scatta il nuovo tetto Isee per otten...","Da oggi, mercoledì primo gennaio, a chi ha dir...",2020-01
2020-01-01,"«Il mio cinema ha avuto successo, ma non ho ma...",«Ho due progetti nella testa e nel cuore. Il p...,2020-01
2020-01-01,"«Il mio cinema ha avuto successo, ma non ho ma...",«Ho due progetti nella testa e nel cuore. Il p...,2020-01
...,...,...,...
2020-07-14,L’età dell’Odio in rete e la ricerca tribale d...,Perché ogni giorno internet sembra darci in pa...,2020-07
2020-07-14,Il lavoro (quando c’è) fa scattare il richiamo...,I lavoratori fanno la fila per i posti di lavo...,2020-07
2020-07-14,"Bmw iX3, ecco il primo suv elettrico bavarese",La iX3 apre la strada alla nuova generazione d...,2020-07


In [ ]:
#regular expression: formatting text
def formatting(texts):
    texts = [regex.sub('\s+', ' ', text) for text in texts] #remove new lines
    texts = [regex.sub("\"", "", text) for text in texts] #remove quotations
    texts = [regex.sub("\'", "", text) for text in texts] #remove quotations
    return texts
formatted_text = formatting(texts)

In [ ]:

import it_core_news_sm
stop_words = list(ita)

url = "https://countwordsfree.com/stopwords/italian/txt"
file = urllib.request.urlopen(url)
stop_ext = [line.decode("utf-8").strip() for line in file]
stop_words = stop_words + stop_ext
stop_words.extend(['q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m'])
stop_words.extend(['calere','indicatori_re','gennaio','febbraio','marzo','aprile','maggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre','ce'])
lemma = it_core_news_sm.load()

def token(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) 

def bigram_trigram(texts):
    bigram = gensim.models.Phrases(texts)
    trigram = gensim.models.Phrases(bigram[texts])
    bigram_ = gensim.models.phrases.Phraser(bigram)
    trigram_ = gensim.models.phrases.Phraser(trigram)
    texts = [[word for word in doc if word not in stop_words] for doc in texts] 
    bi = [bigram_[text] for text in texts]
    tri = [trigram_[bigram_[b]] for b in bi]
    return tri
def processing(texts, stop_words=stop_words):
    texts = [[word for word in doc if word not in stop_words] for doc in texts]
    output = []
    for text in texts:
        doc = lemma(" ".join(text)) 
        output.append([token.lemma_ for token in doc])
    # remove stopwords once more after lemmatization
    output = [[word for word in doc if word not in stop_words] for doc in output]
    return output

tokens = list(token(formatted_text))
bi_tri = bigram_trigram(tokens)
final_data = processing(bi_tri)
df['ProcessedArticles'] = final_data
# df

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def search_risk(text,n):
    word = r'\W*([\w]+)'
    rischio = regex.findall(r'{}\W*{}{}'.format(word*n,'rischio',word*n), text)
    rischiare = regex.findall(r'{}\W*{}{}'.format(word*n,'rischiare',word*n), text)
    incertezza = regex.findall(r'{}\W*{}{}'.format(word*n,'incertezza',word*n), text)
    paura = regex.findall(r'{}\W*{}{}'.format(word*n,'paura',word*n), text)
    timore = regex.findall(r'{}\W*{}{}'.format(word*n,'timore',word*n), text)
    return rischio+rischiare+incertezza+paura+timore


In [28]:
risk = [(search_risk(' '.join(data),10)) for data in final_data ]

In [29]:
df['Risk'] = risk

In [39]:
stop_words = stop_words + stop_ext
stop_words.extend(['q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m'])
stop_words.extend(['calere','indicatori_re','gennaio','febbraio','marzo','aprile','maggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre','ce','irlanda','coronavirus_ncov'])

def frequency(data,top_N,term):
  freq = []
  for i in df['weekoftheyear'].unique():
    week_data = df[(df['weekoftheyear'] == i)][term]
    flat_list = [item for sublist in week_data for item in sublist]
    flat_list1 = [item for sublist in flat_list for item in sublist]
    output = [word for word in flat_list1 if word not in stop_words]
    word_dist = nltk.FreqDist(output)
    temp = pd.DataFrame(word_dist.most_common(),
                      columns=['Word', 'Frequency'])
    temp['Percentage'] = temp.Frequency / len(temp.Word) * 100
    freq.append(temp.head(20))
  return freq


In [40]:
risk = frequency(df['ProcessedArticles'],20,'Risk')



In [ ]:
#  for i in range(len(risk)): 
#   try:
#     word_dict = dict(zip(risk[i].Word, risk[i].Frequency))

#     wordcloud = WordCloud(background_color="white", width=900,height=500, max_words=1628,relative_scaling=1,normalize_plurals=False).generate_from_frequencies(word_dict)
#     plt.imshow(wordcloud)
#     print(('WEEK===' + str(i+1)))
#     plt.axis("off")
#     plt.show()
#   except:
#     print('no data')

In [32]:
# df = df.assign(weekoftheyear=pd.PeriodIndex(df.index, freq='W'))
df

,Title,Article,weekoftheyear,ProcessedArticles,Risk
Date,,,,,
2020-01-01,Europa e la cultura: il bilancio 2019,Il Parlamento Europeo ha recentemente sospeso ...,2019-12-30/2020-01-05,"[parlamento_europeo, sospendere, negoziato, bi...","[(museo, italiano, sapere, rivolgersi, farlo, ..."
2020-01-01,L’anno dei “lateral hire”: quando il socio del...,Il 2019 è stato un anno record per quanto rigu...,2019-12-30/2020-01-05,"[record, riguardare, movimento, avvocato, soci...",[]
2020-01-01,"Bollette, scatta il nuovo tetto Isee per otten...","Da oggi, mercoledì primo gennaio, a chi ha dir...",2019-12-30/2020-01-05,"[mercoledi_gennaio, diritto, cosiddetto_bonus,...",[]
2020-01-01,"«Il mio cinema ha avuto successo, ma non ho ma...",«Ho due progetti nella testa e nel cuore. Il p...,2019-12-30/2020-01-05,"[progetto, testare, cuore, riguardare, temere,...",[]
2020-01-01,"«Il mio cinema ha avuto successo, ma non ho ma...",«Ho due progetti nella testa e nel cuore. Il p...,2019-12-30/2020-01-05,"[progetto, testare, cuore, riguardare, temere,...",[]
...,...,...,...,...,...
2020-07-14,L’età dell’Odio in rete e la ricerca tribale d...,Perché ogni giorno internet sembra darci in pa...,2020-07-13/2020-07-19,"[internet, darci, pasto, capro_espiatorio, dif...","[(dallalto, basso, filosofico, colloquiale, co..."
2020-07-14,Il lavoro (quando c’è) fa scattare il richiamo...,I lavoratori fanno la fila per i posti di lavo...,2020-07-13/2020-07-19,"[lavoratore, filo, sud, trovare, trovare, rego...",[]
2020-07-14,"Bmw iX3, ecco il primo suv elettrico bavarese",La iX3 apre la strada alla nuova generazione d...,2020-07-13/2020-07-19,"[ix, aprire, strada, nuova_generazione, bmw, f...",[]


In [41]:
df_risk = []
for i in range(len(risk)):
  a = (risk[i].Word.head(5).to_list())
  df_risk.append(a)
df_risk = pd.DataFrame(df_risk,columns=['Word1','Word2','Word3','Word4','Word5'])
# df_risk.to_csv('/content/gdrive/My Drive/Research/Datasets/risk_ita2020all.csv')
print(df_risk.to_latex())


\begin{tabular}{llllll}
\toprule
{} &        Word1 &        Word2 &        Word3 &       Word4 &         Word5 \\
\midrule
0  &    sicurezza &  confliggere &       potere &     mercato &       mettere \\
1  &      mercato &      mettere &      settore &      dovere &      politico \\
2  &      mercato &       debito &       dovere &      potere &  investimento \\
3  &       potere &       italia &     politico &   impattare &    processare \\
4  &  coronavirus &         cina &       potere &     azienda &  investimento \\
5  &  coronavirus &    rischiare &      mercato &      dovere &        cinese \\
6  &  coronavirus &         cina &       cinese &     mercato &  approfondire \\
7  &  coronavirus &        virus &         cina &   infezione &    condizione \\
8  &  coronavirus &   contagiare &       potere &      dovere &    situazione \\
9  &  coronavirus &   contagiare &       italia &     mercato &      italiano \\
10 &  coronavirus &       potere &       dovere &   rischiare &    

In [ ]:
i=3
for week in df.index:
  if week.weekofyear == i:
    print(week)
    i=i+1

2020-01-19 00:00:00
2020-01-21 00:00:00
2020-01-27 00:00:00
2020-02-03 00:00:00
2020-02-10 00:00:00
2020-02-17 00:00:00
2020-02-24 00:00:00
2020-03-02 00:00:00
2020-03-09 00:00:00
2020-03-16 00:00:00
2020-03-23 00:00:00
2020-03-30 00:00:00
2020-04-06 00:00:00
2020-04-13 00:00:00
2020-04-20 00:00:00
2020-04-27 00:00:00
2020-05-04 00:00:00
2020-05-11 00:00:00
2020-05-18 00:00:00


In [ ]:
df1 = pd.read_csv('/content/gdrive/My Drive/Research/final_risk_10_ITA.csv')
df1.head()

,Date,Article,Title,weekoftheyear,ProcessedArticles,Risk
0,2020-01-19,Aumenta la preoccupazione per l’ondata di polm...,Timori per il virus polmonare cinese: i casi p...,3,"['aumentare', 'preoccupazione', 'ondata', 'pol...","[('polmonite', 'cina', 'causare', 'ceppo', 'co..."
1,2020-01-21,NEW YORK - Gli Stati Uniti riportano il primo ...,"Virus cinese, primo caso negli Usa. Riunione d...",4,"['new_york', 'unito', 'riportare', 'coronaviru...","[('cdc', 'riferire', 'mass', 'medio', 'persona..."
2,2020-01-21,I timori per il contagio del coronavirus in Ci...,"Europa limita perdite sul finale, moda e lusso...",4,"['timore', 'contagiare', 'coronavirus', 'cina'...","[('sulleconomia', 'atteso', 'consentire', 'fra..."
3,2020-01-21,"Vendite sui titoli del lusso, che risentono de...",Il lusso sotto pressione in Borsa per il virus...,4,"['vendita', 'lussare', 'risentire', 'timore', ...","[('europa', 'kering', 'parigi', 'cattivo', 'ca..."
4,2020-01-22,"L’Oms ha ritenuto opportuno, per ora, non dich...",Virus cinese: sembra escluso il caso del conta...,4,"['oms', 'ritenere', 'opportuno', 'dichiarare',...","[('medico', 'loms', 'chiedere', 'cina', 'conti..."


In [ ]:
df_risk

,Word1,Word2,Word3,Word4,Word5
2009-12-28/2010-01-03,None,None,None,None,None
2010-01-04/2010-01-10,azienda,referendum,confino,crisi,mettere
2010-01-11/2010-01-17,None,None,None,None,None
2010-01-18/2010-01-24,None,None,None,None,None
2010-02-01/2010-02-07,None,None,None,None,None
...,...,...,...,...,...
2012-09-17/2012-09-23,None,None,None,None,None
2012-10-01/2012-10-07,None,None,None,None,None
2012-11-26/2012-12-02,None,None,None,None,None
2012-12-03/2012-12-09,None,None,None,None,None


In [ ]:
def f(x):
     return pd.Series(dict(A = x['A'].sum(), 
                        B = x['B'].sum(), 
                        C = "{%s}" % ', '.join(x['C'])))

In [ ]:
a = df.groupby(pd.Grouper(freq='M'))['ProcessedArticles'].sum()

In [ ]:
df = pd.DataFrame(a)

In [ ]:
final_data

In [43]:
df1 = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/ita2020.csv')
df1.groupby('Date').count()

,Title,Article
Date,,
2020-01-01,17,17
2020-01-02,33,33
2020-01-03,31,31
2020-01-04,31,31
2020-01-05,11,11
...,...,...
2020-07-10,63,63
2020-07-11,34,34
2020-07-12,11,11


In [44]:
len(final_data)

8175

In [52]:
risk  = []
for data in final_data:
  if ('rischio'  or 'paura' or 'timore' or 'incertezza') in data:
    risk.append(data)
  else:
    risk.append(None)

In [53]:
df1['countrisk'] = risk

In [56]:
df1.dropna().groupby('Date').count()

,Title,Article,countrisk
Date,,,
2020-01-02,4,4,4
2020-01-06,3,3,3
2020-01-07,1,1,1
2020-01-08,1,1,1
2020-01-09,1,1,1
...,...,...,...
2020-07-09,9,9,9
2020-07-10,4,4,4
2020-07-12,2,2,2
